# Lab 7: Bayesian Hierarchical Modeling

### Lab Date: Wednesday, Mar 12

### Lab Due: Wednesday, Apr 2

## Instructions

Work with your lab group to complete the following notebook. Your work will be reviewed by your peers in three weeks (Wednesday, April 2nd)

In this lab, you will:
1. Understand a Bayesian hierarchical model in the context of image reconstruction.
2. Apply the Fourier transform as a transformation matrix in an inverse problem.
3. Implement Maximum A Posteriori (MAP) estimation using:
	- Coordinate Ascent Optimization
	- Gradient-Based Optimization (L-BFGS-B)
4. Compute and interpret Laplace Approximation to estimate posterior uncertainty.
5. Compare the effectiveness of different optimization methods in Bayesian inference.

This lab applies Bayesian sparse regression to the MNIST dataset, where images of handwritten digits are processed and reconstructed using probabilistic models.

If you are new to working in python, or in a Jupyter notebook, please ask your lab members for help. If you notice a lab member struggling, and have experience, please offer your help.

Please see this [Ed post](https://edstem.org/us/courses/74615/discussion/6352659) for corrections, questions, and discussion. If you would rather work with your own copy of the files, I have uploaded a zip folder there with the lab materials. 

Corrections to the lab will be pushed directly to this notebook. We will only push corrections to the text, which is set to read only to prevent merge conflicts. In the event of a merge conflict, save your notebook under a different name, and click the link that launches the lab from the schedule on the [stat238 homepage](https://stat238.berkeley.edu/spring-2025/) again. Then, check for discrepancies. If you can't find them, or resolve the conflict, contact us.

## Set Up:

Run the cells below to load necessary packages and to load the example data set.

In [ ]:
# If torchvision does not load for you, uncomment the line below and install torchvision
# %pip install torch torchvision

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets 
import scipy
from scipy.linalg import inv
from scipy.stats import invgamma
from scipy.optimize import minimize

In [ ]:
# Load an MNIST digit as an example
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((28, 28))])
mnist_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)

# Select an image from the dataset
img, label = mnist_dataset[0]  # Get the first image
img_array = img.squeeze().numpy()  # Convert from PyTorch tensor to NumPy
x_true = img_array.flatten()  # Flatten the image for processing

## Introduction

Image reconstruction is a common problem in fields such as medical, geophysical, and astronomical imaging. In this lab, we will explore an application of a Bayesian hierarchical model to reconstruct an image that has been blurred by a low-pass filter (a filter that damps high-frequency components of the image) and that has been corrupted by noise.

We assume that the true image $x$ is transformed by an operator $A$, and we observe $\tilde{x}$:

$$\tilde{x} = A x + \epsilon, \quad \epsilon \sim \mathcal{N}(0, \sigma^2 I)$$

where $A$ represents the low-pass filter. 

Any low pass filter can be decomposed $A = F^{*} D F$ where $F$ is a Fourier matrix (the matrix that implements a discrete Fourier transform), and $D$ is a diagonal matrix with decreasing non-negative entries. Since the discrete Fourier transform is orthonormal, its conjugate transpose, $F^{*}$, is $F^{-1}$, so $D$ is the matrix of eigenvalues of the low-pass filter, and $F$ contains its eigenvectors. Since $F$ is unitary, $\zeta = F \epsilon \sim \mathcal{N}(0,\sigma^2 F F^*) = \mathcal{N}(0,\sigma^2 I)$. 

Since most signal processing pipelines begin by converting the original received signal, $\tilde{x}$, to its Fourier transform, $y = F \tilde{x}$, we can assume that the problem takes the form:

$$ \text{Estimate  } x \text{  given: } \quad y = A x + \zeta, \quad A = D F, \quad \quad \zeta \sim \mathcal{N}(0, \sigma^2 I).$$

To recover $x$, we will try a Bayesian hierarchical model. First, we assume that the elements of $x$ are exchangeable, so they may be modeled as conditionally i.i.d. Here, we assume that the entries of the true image are, conditional on an unknown scale parameter, drawn i.i.d. from a Gaussian prior:

$$x_i \sim \mathcal{N}(0, \tau^2)$$

We place an Inverse-Gamma hyperprior on $\tau^2$:

$$\tau^2 \sim \text{Inv-Gamma}(\alpha, \beta)$$

Our goal is to estimate $x$ using MAP estimation and compare different optimization methods. We will estimate $x$ and $\tau$ jointly. This will, in effect, place an $L_2$ regularization on the image recovery problem where the regularization parameter is unknown and optimized in tandem with the image $x$. 

## Part I: Model Construction

With your group, make a diagram representing the hierarchical model. Then, answer the questions below.

**Q 1.1:** Find the marginal prior distribution for each $x_i$ by marginalizing over $\tau^2$. Show that this mixture produces a standard distribution. Hint: it will help to know that, an inverse-gamma distributed random variable is the reciprocal of a Gamma distributed random variable with the same parameters. What does this result suggest about the regularizing effect of the hierarchical model?

*Write your answer here*

**Q 1.2:** Under this model, are the entries of the true image i.i.d.? Are they correlated?

*Write your answer here*

**Q 1.3:** Write a code that can sample $x_1, x_2$ from the hierarchical model. Produce 1,000 samples, $\{x_1(i),x_2(i)\}_{i=1}^{1,000}$, and make a scatter plot showing the position of the samples. Make sure to match the aspect ratio of the axes in your plot.

In [ ]:
# Insert your sampling code

**Q 1.4:** Note any patterns you observe in the scatter plot. In particular, comment on how $x_1$ and $x_2$ are coupled by the hierarchical model. In what sense do they share information?

*Write your answer here*

**Q 1.5:** Suppose we modified the hierarchical model so that, instead of drawing a global scale parameter $\tau^2$, shared by all $x_i$, we drew a scale for each $x_i$, $\tau_i^2$, i.i.d. from an inverse gamma with parameters $\alpha$ and $\beta$. This is sometimes called "local regularization". Sketch the ensuing hierarchical model, then discuss how switching from global to local regularization would change the way information is shared across the model, and change the character of solutions promoted by the prior if we optimize over $x$ and $\tau$ jointly.

*Write your answer here*

### Construct the Low-Pass Filter

In [3]:
def fourier_transform_matrix(size):
    """Create a Fourier transformation matrix."""
    F1 = scipy.linalg.dft(size)/np.sqrt(size) # Build orthonormal discrete-Fourier transform matrix in one dimension
    F2 = np.kron(F1, F1) # extend to the two-dimensional transform
    return F2

# Define the low-pass filter
def low_pass_filter(size):
    """Create the low pass filter."""
    d = 10**np.linspace(0,-1.5,size) # notice, this is chosen so that the eigenvalues of A cover many orders of magnitude
    D1 = np.diag(d)
    D = np.kron(D1,D1)
    F = fourier_transform_matrix(size)
    A = D @ F
    return A

In [ ]:
# Visualize transforms
plt.figure(figsize=(25, 5))

F = fourier_transform_matrix(12)
A = low_pass_filter(12)

plt.subplot(1, 4, 1)
plt.imshow(F.real, cmap="gray")
plt.title("Fourier Matrix (real)")

plt.subplot(1, 4, 2)
plt.imshow(F.real, cmap="gray")
plt.title("Fourier Matrix (imag)")

plt.subplot(1, 4, 3)
plt.imshow(A.real, cmap="gray")
plt.title("Filter (real)")

plt.subplot(1, 4, 4)
plt.imshow(A.real, cmap="gray")
plt.title("Filter (imag)")

plt.show()

### Simulate the Observations
1. Select an MNIST image and flatten it into a vector $x_{\text{true}}$.
2. Create the transformation matrix $A$.
3. Generate observations by applying $A$ to $x_{\text{true}}$, and then by adding noise to the transformed image.

In [ ]:
# Define noise level
noise_std = 0.05  # Standard deviation of Gaussian noise

# Apply the transformation matrix and add noise
A = low_pass_filter(len(x_true))
y = A @ x_true + np.random.normal(0, noise_std, size=x_true.shape)  # Noisy observations

## Part II: Point Estimation

To try and recover $x$, from $y$, we will try to find the joint MAP estimator $\{\hat{x}(y),\hat{\tau}^2(y)\}$.

**Q 2.1:** With your group members discuss whether the joint MAP problem, find $\{\hat{x}(y),\hat{\tau}^2(y)\}$ that maximize $p_{x,\tau|y}$, is distinct from the marginal MAP problem, find $\{\hat{x}(y)\}$ that maximize $p_{x|y}$. Does the choice to maximize with, or marginalize over, the nuisance parameter $\tau$ change the nature of the estimator? Comment on which of these two procedures would be easier to implement computationally for general hierarchical models.

*Write your answer here.*

### Coordinate-Ascent

First, we will consider a coordinate ascent scheme. Coordinate ascent is an optimization technique that iteratively optimizes distinct subsets of the variables while keeping others fixed. For example, we might optimize $x_1$ holding all other variables fixed, then optimize $x_2$ holding all other variables fixed, and so on. The general algorithm is available in https://en.wikipedia.org/wiki/Coordinate_descent.

Coordinate ascent is commonly used in problems where the full optimization problem is hard to solve directly but can be broken down into simpler subproblems. This is often the case for hierarchical models built out of tractable components. For example, it is common to use conditionally Gaussian hierarchical models, with independent elementwise priors on the parameters of the Gaussian. Then, each step that pools over related variables can be optimized in closed form via the linear algebra of the normal, normal model. The remaining variables may be optimized independently, in parallel. This breaks the problem into a recursive procedure that solves a sequence of easy optimize problems (e.g. least squares for pooling information in multiple dimensions, then a series of one-dimensional optimization problems). This approach retains expressivity through the mixtures encoded in the hierarchical model. For example, $L_1$ regularization may be represented as the MAP solution to a hierarchical model, for which the standard LASSO solution is produced using a coordinate ascent algorithm on the individual $x_i$ jointly with a local scale parameter. The order, and subsets of variables optimized simultaneously, are up to the designer to pick. Usually, these should be chosen to mirror the architecture of the hierarchical model.

In our case, we seek to maximize the posterior distribution of $x, \tau$ given $y$. We will, alternately, optimize $x$ given $\tau$, then optimize $\tau$ given $x$.


#### 1. Updating $x$

In the first stage of coordinate ascent, we fix $\tau$, and optimize over $x$.

**Q 2.2:** Show that, finding the conditional MAP estimator of $x$ given $\tau$ is equivalent to minimizing:

$$f(x|\tau,y) = \| A x - y \|^2 + \frac{1}{2\tau^2} \|x\|^2$$

where $a_i$ is the $i$-th column of $A$.

*Write your answer here.*

**Q 2.3:** Show that the conditional MAP estimator of $x$ given $\tau$ is given by (check HW 3, or the normal, normal exercises in lab 3):

$$\hat{x}(\tau,y) = \left(A^{*} A + \frac{1}{2 \tau^2} I \right)^{-1} A^{*} y $$

*Write your answer here.*

This suggests an automatic update step for $x$ given $\tau$. If $\{(x^{(t)},\tau^{(t)})\}_{t=0}^{...}$ is a sequence of guesses for the MAP estimator that iteratively approach the MAP via coordinate ascent, then, when updating the image $x$, given the scale $\tau$, we should set:

$$x^{(t+1)} = \left(A^* A +  \frac{1}{2 {\tau^2}^{(t)}} I \right)^{-1} A^* y $$

#### 2. Updating $\tau^2$

In the second stage, we optimize over $\tau$, holding $x$ fixed. That is, we aim to solve analytically for the conditional MAP estimator of $\tau$ given $x$.

**Q 2.4:** Using the structure of the hierarchical model, argue that the MAP estimator for $\tau$ given $x$ is independent of $y$. 

*Write your answer here.*

**Q 2.5:** To find the MAP estimator for $\tau$ given $x$, show that $\tau^2|x$ has posterior density:

$$p(\tau^2 | x) \propto (\tau^2)^{-(\alpha + n/2 + 1)} \exp\left(-\frac{\beta + 0.5 \|x\|^2}{\tau^2}\right)$$

which is maximized at:

$$\hat{\tau}^2_{\text{MAP}}(x) = \frac{\beta + \tfrac{1}{2} \|x\|^2}{\alpha + \tfrac{1}{2}n + 1}$$

where $n$ is the number of entries in $x$.

*Write your answer here.*

**Q 2.6:** Refer back to Part I. Does this updating procedure match your intuition about how information should be passed through the model?

*Write your answer here.*

This suggests the recursive step:

$${\tau^2}^{(t+1)} = \frac{\beta + \tfrac{1}{2} \|x^{(t+1)}\|^2}{\alpha + \tfrac{1}{2}n + 1}$$

**Q 2.7:** How would this update step change if we adopted the second hierarchical model described in Part I (that is, used an indepndent scale parameter for each $x_i$)?

*Write your answer here.*

#### 3. Implementation

The full recursion is now:

$$(x^{(t)},{\tau^2}^{(t)}) \longrightarrow x^{(t+1)} = \left(A^* A +  \frac{1}{2 {\tau^2}^{(t)}} I \right)^{-1} A^* y \longrightarrow {\tau^2}^{(t+1)} = \frac{\beta + \tfrac{1}{2} \|x^{(t+1)}\|^2}{\alpha + \tfrac{1}{2}n + 1} \longrightarrow (x^{(t+1)},{\tau^2}^{(t)}) $$

1. Implement coordinate ascent to iteratively update each variable $X_i$.
2. Update the hyperparameter $\tau^2$ using an Inverse-Gamma posterior.

In [5]:
# Implement Coordinate Ascent
def coordinate_ascent_fourier(A, y, alpha=2.0, beta=2.0, tol=1e-6, max_iter=300):
    """
    Perform MAP estimation for Bayesian hierarchical model using Coordinate Ascent.
    - X has a Gaussian prior: X_i ~ N(0, τ^2)
    - τ^2 has an Inverse-Gamma hyperprior
    """
    n = A.shape[1]
    x = np.zeros(n)  # Initialize X
    tau2 = 0.5  # Initial value of tau2

    AstarA = A.conj().T @ A
    Astary = A.conj().T @ y

    for iter in range(max_iter):
        x_old = x.copy()  # Store previous x for convergence check

        # The code below updates x using a coordinate ascent scheme.
        #for i in range(n):
        #    ai = A[:, i]
        #    residual = Y - A @ X + ai * X[i]
        #    X[i] = (ai @ residual) / (ai @ ai + 1/tau2 + 1e-6)  # Ensure numerical stability

        # Update x given τ² (note: it is good numerical practice to avoid forming inverses. 
        # There are stabler and faster ways to solve linear systems. These are implemented in most solvers)
        x = np.solve(AstarA + (1/(2*tau2))*np.eye(n), Astary)
        
        # Update τ² given x
        tau2 = (beta + 0.5 * np.sum(X**2)) / (alpha + n/2 + 1)

        # Check for convergence
        if np.linalg.norm(x - x_old) < tol:
            break

    return x, tau2

### Gradient-Based Optimization (L-BFGS-B)

Alternately, we could optimize the posterior with respect to $x$ and $\tau$ using a standard gradient-based optimizer (e.g. gradient descent, Newton's method, etc).

**Q 2.8:** Show that finding $\hat{x}_{\text{MAP}}(y),\hat{\tau}_{\text{MAP}}^2(y)$ is the same as finding $\hat{x}$, $\hat{\tau}^2$ that jointly maximize the objective:

$$f(x, \tau^2| y) = \| A x - y \|^2 + \frac{1}{2\tau^2} \|x\|^2+ \alpha \log(\tau^2) + \frac{\beta}{\tau^2}.$$

*Write your answer here.*

**Q 2.9:** As is often true for hierarchical models built out of tractable parts, all derivatives of the log posterior are available in closed form. These can be passed as functions to an optimizer (e.g. pass a gradient and a Hessian for Newton type methods). As an example, compute the gradient of $f$ with respect to $x$ and $\tau$ explicitly.

*Write your answer here.*

The code below minimizes this objective with a standard, gradient-based optimizer.

In [6]:
# Define the objective function
def objective_function(params, A, y, alpha=2.0, beta=2.0):
    """Objective function for MAP estimation using a gradient-based optimizer."""
    n = A.shape[1]
    x = params[:-1]  # Extract X from parameter vector
    tau2 = np.exp(params[-1])  # Ensure tau2 is positive by optimizing its log value

    loss = np.linalg.norm(A @ x - y) ** 2 + np.sum(x**2) / (2 * tau2)
    loss += alpha * np.log(tau2) + beta / tau2  # Inverse-Gamma prior on tau2
    return loss

# Optimize using L-BFGS-B
def gradient_based_optimizer(A, y):
    """Optimize the MAP estimate using a gradient-based optimizer, jointly optimizing X and tau2."""
    n = A.shape[1]
    x_init = np.zeros(n)
    tau2_init = np.log(0.5)  # Optimize log(tau2) to enforce positivity

    params_init = np.concatenate([x_init, [tau2_init]])

    # Optimize both X and tau2
    result = minimize(objective_function, params_init, args=(A, y), method='L-BFGS-B')
    
    x_opt = result.x[:-1]
    tau2_opt = np.exp(result.x[-1])  # Convert back from log space
    return x_opt, tau2_opt

## Part III: Laplace Approximation for Uncertainty Estimation

Suppose that we've run either coordinate-ascent or BFGS to convergence, and they have returned an estimator $\hat{x}_{\text{MAP}}(y), \hat{\tau}^2_{\text{MAP}}(y)$. 

**Q 3.1:** Derive the Hessian of the log-posterior density about $\hat{x}, \hat{\tau}$. Name it $H(\hat{x},\hat{\tau})$, and show it only depends on the data $y$ through its arguments $\hat{x}, \hat{\tau}$.


*Write your answer here.*

**Q 3.2:** Argue that, under a joint Laplace approximation to the posterior over $x$ and $\tau$, the Laplace approximation to the marginal posterior distribution of $x$ (marginalizing over $\tau$) is also normal, with covariance given by dropping one row and column from $H(\hat{x},\hat{\tau})$. 

*Write your answer here.*

**Q 3.3:** Complete the code below to return the covariance of the Laplace approximation to the posterior given a joint MAP estimator $\hat{x}, \hat{\tau}^2$.

In [7]:
# Compute Laplace Approximation based on posterior
def laplace_approximation_covariance(A, x_map, tau2, sigma2=1.0):
    """Compute the Laplace approximation to the posterior."""
    # compute the Hessian of the log-posterior
    # invert to estimate the covariance (I'd suggest adding a small multiple of the identity to the Hessian before inverting for stability).
    return covariance

## Part IV: Visualization and Comparison

Let's compare our results. First, we will plot the MAP estimates for the images produced by both algorithms. Then, (*optional*) we will estimate the uncertainty in those estimates using our Laplace approximation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv

# Compute MAP estimates using Bayesian hierarchical model with Fourier transform
x_map_fourier, tau2_estimate = coordinate_ascent_fourier(A, y)
x_map_gradient, tau2_gradient = gradient_based_optimizer(A, y)

# Compute Laplace Approximation with posterior-based covariance
x_map, covariance = laplace_approximation(A, x_map_fourier, tau2_estimate)

# Reshape for visualization
x_map_fourier_img = x_map_fourier.reshape(28, 28)
x_map_gradient_img = x_map_gradient.reshape(28, 28)

# --- Visualization ---
plt.figure(figsize=(25, 5))

# Original MNIST Image
plt.subplot(1, 4, 1)
plt.imshow(img_array, cmap="gray")
plt.title("Original MNIST Image")

# Fourier Transformed Image (Log Magnitude for Better Visualization)
plt.subplot(1, 4, 2)
plt.imshow(np.log(1 + np.abs(y.reshape(28, 28))), cmap="gray")
plt.title("Observed Image y")

# Reconstructed Image using Coordinate Ascent
plt.subplot(1, 4, 3)
plt.imshow(x_map_fourier_img, cmap="gray")
plt.title("Reconstructed (Coord. Ascent)")

# Reconstructed Image using Gradient-Based Optimization
plt.subplot(1, 4, 4)
plt.imshow(x_map_gradient_img, cmap="gray")
plt.title("Reconstructed (Grad-Based)")

plt.show()

**Q 4.1:** In the space below report the MAP estimator for the nuisance parameter $\tau^2$, and a 95\% interval estimate for $\tau^2$ based on the Laplace approximation to the posterior, for both optimization procedures. Are your answers consistent? If not, propose a reason the optimizers may have returned different answers.

In [ ]:
# Insert your code here

**Q 4.2:** *Optional.* To visualize the uncertainty in $x$, do the following:

1. Make a heatmap image showing the standard deviation in each entry of $x$ using the Laplace approximation associated with each approximation to the MAP.
2. Compute the SVD of the covariance of each Laplace approximation. The singular vectors of the covariance are the principal components. Make a series of 6 heatmap images, 3 per optimization method, each showing the 1st, 2nd, and 3rd principal components of the Laplace covariance. To do this, extract the first three singular vectors of the covariance, then reshape each to make an image. You may borrow some of the code written above to help visualize the results. Comment on the directions of principal uncertainty in the posterior.

In [ ]:
# Insert your code here

**Q 4.3:** Comment on the MAP estimators for the images and scale. If completed, the directions of principal posterior uncertainty about the estimator. How do you expect your results would change if you used a set of unknown local scale parameters $\{\tau_i\}_{i=1}^{...}$ instead of a shared, global scale parameter? What signals $x$ would be promoted by this prior model? Would posterior estimation remain stable with twice as many unknowns as data points and as many nuisance parameters as parameters of inferential interest?

*Write your answer here.*